In [0]:
!pip install q keras==2.3.0
!pip install tensorflow_hub

In [0]:
!pip install tensorflow==2.1.0

In [3]:
from __future__ import absolute_import, division, print_function, unicode_literals
import matplotlib.pylab as plt

%tensorflow_version 2.1.0
import tensorflow as tf

import tensorflow_hub as hub
from keras.models import Sequential
from keras.layers.core import Dense
from keras.optimizers import SGD

import numpy as np
import pandas as pd

from sklearn.preprocessing import LabelEncoder
from keras.utils import np_utils
import pickle
import cv2
import os
import h5py
import keras
print(keras.__version__)
print(tf.__version__)

`%tensorflow_version` only switches the major version: 1.x or 2.x.
You set: `2.1.0`. This will be interpreted as: `2.x`.


TensorFlow 2.x selected.


Using TensorFlow backend.


2.3.0
2.1.0


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
hdf5_path = '/content/drive/My Drive/Audio_Quality_Analysis/data/dataset_2_hdf5/Audio_Classification_Train_HDF5_0423_v1_.hdf5'
dataset = h5py.File(hdf5_path, "r")

In [6]:
train_labels=np.array(dataset["train_labels"])
train_labels = train_labels.reshape((len(train_labels),-1))

print(len(train_labels))

le = LabelEncoder()
train_labels = le.fit_transform(train_labels)
labels = np_utils.to_categorical(train_labels, 2)

with open('/content/drive/My Drive/Audio_Quality_Analysis/model/labelEncoderFile_hdf5_04232020_v1.pkl', 'wb') as fid:
    pickle.dump(le, fid)

print(train_labels[0])
print(le.classes_)

11191
1
[0 1]


/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_label.py:251: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [0]:
INIT_LR = 1e-1
BS = 8
EPOCHS = 50

In [0]:
train_batch_imgs=[]
for i in range(10):
   img=(dataset['train_img'])[i]
   img=img/255.
   train_batch_imgs.append(img)    
train_batch_imgs=np.array(train_batch_imgs)

In [17]:
from sklearn.model_selection import train_test_split
(trainX, testX, trainY, testY) = train_test_split(train_batch_imgs, labels, test_size=0.25, random_state=42)
trainX[0].shape

(224, 224, 3)

In [19]:
print("[INFO] performing 'on the fly' data augmentation")
from keras.preprocessing.image import ImageDataGenerator
aug = ImageDataGenerator(
    rotation_range=20, zoom_range=0.15, width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.15,
    horizontal_flip=True,
    fill_mode="nearest")

[INFO] performing 'on the fly' data augmentation


In [20]:
model = tf.keras.Sequential([
                    hub.KerasLayer(
                        "https://tfhub.dev/google/tf2-preview/mobilenet_v2/feature_vector/4",
                        output_shape=[1280], trainable=False
                    ),
                    tf.keras.layers.Dropout(0.4),
                    tf.keras.layers.Dense(2, activation='softmax')
                    ])

model.build([None, 224, 224, 3])
model.summary()

model.compile( optimizer=tf.keras.optimizers.Adam(), loss='categorical_crossentropy', metrics=['acc'])

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer (KerasLayer)     multiple                  2257984   
_________________________________________________________________
dropout (Dropout)            multiple                  0         
_________________________________________________________________
dense (Dense)                multiple                  2562      
Total params: 2,260,546
Trainable params: 2,562
Non-trainable params: 2,257,984
_________________________________________________________________


In [21]:
print("[INFO] training network for {} epochs...".format(EPOCHS))
H = model.fit_generator(
    aug.flow(trainX, trainY, batch_size=BS),
    validation_data=(testX, testY),
    steps_per_epoch=len(trainX) // BS,
    epochs=EPOCHS)

print(H)

model.save("/content/drive/My Drive/Audio_Quality_Analysis/model/AudioQuality_MobileNet_HDF5_TL_V1_04222.h5")
model.save_weights("/content/drive/My Drive/Audio_Quality_Analysis/model/AudioQualityWeights_MobileNet_HDF5_TL_V1_0422.h5")

[INFO] training network for 2 epochs...
Instructions for updating:
Please use Model.fit, which supports generators.


Instructions for updating:
Please use Model.fit, which supports generators.


Train for 1 steps, validate on 3 samples
Epoch 1/2
1/1 [==============================] - 3s 3s/step - loss: 0.5434 - acc: 0.7143 - val_loss: 0.6445 - val_acc: 0.3333
Epoch 2/2
1/1 [==============================] - 0s 470ms/step - loss: 1.0002 - acc: 0.5714 - val_loss: 0.7987 - val_acc: 0.3333


In [0]:
from keras import models
model = tf.keras.models.load_model('/content/drive/My Drive/Audio_Quality_Analysis/model/AudioQuality_TL_GD_V1_04222.h5',custom_objects={'KerasLayer':hub.KerasLayer})
model.load_weights('/content/drive/My Drive/Audio_Quality_Analysis/model/AudioQualityWeights_TL_GD_V1_0422.h5')
model.summary()

In [0]:
val_image_batch, val_label_batch = next(iter(valid_generator))
true_label_ids = np.argmax(val_label_batch, axis=-1)

print("Validation batch shape:", val_image_batch.shape)

tf_model_predictions = model.predict(val_image_batch)
tf_pred_dataframe = pd.DataFrame(tf_model_predictions)
tf_pred_dataframe.columns = dataset_labels

print("Prediction results for the first elements")
tf_pred_dataframe.head()